In [0]:
import numpy as np
import pandas as pd
import datetime
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#лайфхак, если не тянет компьютер - для расчета можно использовать Google Colab, в моем случае помогло ускориться раз в 5
start_time = datetime.datetime.now()

# 1. Загрузим наш датасет: в X запишем признаки, не связанные с итогами матча, в y - исходы матчей
data = pd.read_csv('./features.csv', index_col='match_id')
X = pd.DataFrame(data.loc[:, 'start_time':'dire_first_ward_time'])
y = pd.DataFrame(data['radiant_win'])

# 2. Узнаем столбцы, в которых есть пропуски и запишем их для удобства в массив nan_columns
nan_columns = np.array([])

for col in data.columns:
  if np.isnan(data[col]).any() == True:
    nan_columns = np.append(nan_columns, col)
    #print(col, data[col].count()) #для просмотра количества непустых значений
    
#Распечатаем наш массив с колонками, содержащими NaN значения, выпишем описания двух признаков
#print(nan_columns)
#first_blood_time - игровое время первой крови
#first_blood_team - команда, совершившая первую кровь
#обоснование для пропущенных значений - события не наступили в течении первых 5 минут

# 3. NaN значения можно заполнить пропущенные значения разными способами:
#    1: заполнить нулями, чтобы данные пропуски не вносили вклад в логистической регрессии
#    2: заполнить значениями, как будто они наступили на 5 минуте, что соответсвует значению 300 (= 60*5)
#    3: заполнить средними значениями
# Попробуем способ 2:
X = X.fillna(300)

# 4. Столбец с целевой переменной - 'radiant_win', мы его загрузили в y

# 5.1 Выберем разбиение:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# 5.2 Обучим наш классификатор и проверим качество на кросс-валидации
# Ниже для простоты приведен код, который обучается на фиксированном значении количества деревьев и скорости обучения
for p in [0.2]:
  clf = GradientBoostingClassifier(n_estimators=500, verbose=False, random_state=42, learning_rate=p)
  clf.fit(X, y)
  c_v = cross_val_score(clf, X, y, cv=kf, scoring='roc_auc')
  print('{c} learning_rate = {p}'.format(c = round(c_v.mean(), 6), p=p))
  print ('Time elapsed:', datetime.datetime.now() - start_time)
  
# 5.3 Ниже приведены время работы и результаты обучения при разном количестве деревьев и показателе скорости обучения
# В предложенных вариантах количества деревьев 10, 20, 30, оптимум однозначно не достигается, так как
# при увеличении количества деревьев, показатель качества на кросс-валидации растет существенно
# В своих расчетах я остановился на количестве деревьев 500, при скорости обучения 0.2, что соответсвует
# разнице с предыдущим шагом меньше 0.001

# Чтобы ускорить обучение при увеличении количества деревьев, можно:
# 1) Брать не всю выборку, а какое-то случайное подмножество
# 2) Можно упростить модель, например, уменьшить глубину деревьев (параметр max_depth)

#n_estimators = 30, время обучения с кросс-валидацией ~ 1m25s
#0.689949 learning_rate = 0.1
#0.698246 learning_rate = 0.2
#0.701622 learning_rate = 0.3
#0.701574 learning_rate = 0.4
#0.701795 learning_rate = 0.5 - максимум
#0.700938 learning_rate = 0.6

#n_estimators = 50, время обучения с кросс-валидацией ~ 2m20s
#0.697816 learning_rate = 0.1
#0.704457 learning_rate = 0.2

#n_estimators = 100, время обучения с кросс-валидацией ~ 5m20s
#0.712547 learning_rate = 0.2
#0.713898 learning_rate = 0.3
#0.71332  learning_rate = 0.4

#n_estimators = 300, время обучения с кросс-валидацией ~ 16m
#0.717853 learning_rate = 0.1
#0.720567 learning_rate = 0.2
#0.718898 learning_rate = 0.3

#n_estimators = 400, время обучения с кросс-валидацией ~ 20m - похоже на 769 результат на kaggle (default gb)
#0.721865 learning_rate = 0.2

#n_estimators = 500, время обучения с кросс-валидацией ~ 24m30s - остановимся на этом шаге (разница с предыдущим меньше 0.001)
#0.722444 learning_rate = 0.2

#стоить заметить, что n_estimators и learning_rate связаны, и определяют
#с какой скоростью мы будем двигаться в сторону антиградиента
#можно зафиксировать один из параметров и варьировать другой

#Проверим наш prediction на kaggle в late submission:
#X_test = pd.read_csv('./features_test.csv', index_col='match_id')
#X_test = X_test.fillna(300)
#pred = pd.Series(clf.predict_proba(X_test)[:, 1], index=X_test.index, name = 'radiant_win')
#X_test = pd.concat([X_test, pred], axis=1)
#res = pd.Series(X_test['radiant_win'], name = 'radiant_win')
#res.to_csv('./submission.csv')
#грузим на kaggle, получаем score = 0.73219 на public leaderbord